# 1. Import and Install Dependencies

In [1]:
!source /home/jean/MEGA/Signify/IA\ model/signify_python/bin/activate
!pip install tensorflow
#!pip install tensorflow-gpu
!pip install opencv-python 
!pip install mediapipe 
!pip install scikit-learn
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
!source /home/jean/MEGA/Signify/IA\ model/signify_python/bin/activate
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [71]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [72]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [73]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [74]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    '''
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=3), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=3)
                             ) 
    '''
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=1)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=1)
                             ) 

# 3. Extract Keypoint Values

In [6]:
#len(results.left_hand_landmarks.landmark)

In [7]:
def extract_keypoints(results):
    #pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    #return np.concatenate([pose, face, lh, rh])
    return np.concatenate([lh, rh])

# 6. Preprocess Data and Create Labels and Features

In [8]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#actions = np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])
actions = np.array(['a','b','c','d'])

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'a': 0, 'b': 1, 'c': 2, 'd': 3}

In [11]:
# Function to read video frames and return an array
def read_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_list = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to the desired format (e.g., height, width, channels)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Adjust channel order if necessary
        frame_list.append(frame)

    cap.release()
    return frame_list

In [17]:
sequences, labels, frames = [], [], []
directory = os.getcwd()
DATA_PATH = os.path.join(directory, 'ASL_Videos')

for action in actions:
    #DATA_PATH = ../ASL_Videos/
    for sequence in [1, 2, 3, 4, 5, 6]:#np.array(os.listdir(os.path.join(DATA_PATH, f'{action}/processed_video'))).astype(int):
        #Looping through LETTER (action) and VIDEO (sequence) number
        #video_path = os.path.join(DATA_PATH, f'{action}/videos/ASL_{action}_{sequence}.mp4')
        sequence_length = os.listdir(os.path.join(DATA_PATH, f'{action}/processed_video/{sequence}'))
        sequence_length = [file for file in sequence_length if file.endswith(".npy")]
        sequence_length = len(sequence_length)
        window = []

        #frame_list = read_video_frames(video_path)
        #frames.append(frame_list)
        
        for frame_num in range(int(sequence_length)):
            #print(f'video {action}-{sequence}, second: {second}')
            #for frame_num in range(30):
                #print(f'frame: {action}-{sequence}-{second}-{frame_num}')
            res = np.load(os.path.join(DATA_PATH, f'{action}/processed_video/{sequence}', f'frame_{frame_num + 1}.npy'))
            window.append(res)            
        sequences.append(window)
        labels.append(label_map[action])

In [18]:
np.array(sequences).shape
#reshape_sequence = np.array(sequences).reshape(np.array(sequences).shape[0], np.array(sequences).shape[1], np.array(sequences).shape[2], 1)

(24, 151, 1662)

In [19]:
np.array(frames).shape

(24, 151, 480, 640, 3)

In [20]:
#np.array(reshape_sequence).shape

In [21]:
np.array(window).shape

(151, 1662)

In [22]:
X = np.array(sequences)

In [23]:
X.shape

(24, 151, 1662)

In [24]:
y = to_categorical(labels).astype(int)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [27]:
y_test.shape

(2, 4)

# 7. Build and Train LSTM Neural Network

In [66]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv3D, Conv1D, MaxPooling3D, Concatenate, MaxPooling1D, LSTM, Dense, BatchNormalization, Flatten, Reshape, Dropout, Lambda, GlobalAveragePooling3D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.regularizers import l2
import tensorflow as tf

In [30]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [39]:
'''
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=( ,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
'''

"\nmodel = Sequential()\nmodel.add(LSTM(64, return_sequences=True, activation='relu', input_shape=( ,1662)))\nmodel.add(LSTM(128, return_sequences=True, activation='relu'))\nmodel.add(LSTM(64, return_sequences=False, activation='relu'))\nmodel.add(Dense(64, activation='relu'))\nmodel.add(Dense(32, activation='relu'))\nmodel.add(Dense(actions.shape[0], activation='softmax'))\n"

In [40]:
model = Sequential()

# 1D Convolutional Layers
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(None, 1662)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten before passing to LSTM
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

# Fully Connected Layers
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))  # Regularization to prevent overfitting
model.add(Dense(actions.shape[0], activation='softmax'))  # Assuming you have 26 ASL signs


2023-10-11 16:23:39.203375: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-11 16:23:39.205317: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [41]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

2023-10-11 16:23:47.416601: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 368964000 exceeds 10% of free system memory.


Epoch 1/500


2023-10-11 16:23:52.948243: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31910400 exceeds 10% of free system memory.
2023-10-11 16:23:52.948290: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31910400 exceeds 10% of free system memory.
2023-10-11 16:23:52.948398: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31910400 exceeds 10% of free system memory.
2023-10-11 16:23:52.948433: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31910400 exceeds 10% of free system memory.


12/12 [==============================] - 6s 74ms/step - loss: 4.8479 - categorical_accuracy: 0.0432
Epoch 2/500
12/12 [==============================] - 1s 98ms/step - loss: 4.7640 - categorical_accuracy: 0.0459
Epoch 3/500
12/12 [==============================] - 2s 145ms/step - loss: 4.5887 - categorical_accuracy: 0.0297
Epoch 4/500
12/12 [==============================] - 2s 167ms/step - loss: 4.4584 - categorical_accuracy: 0.0486
Epoch 5/500
12/12 [==============================] - 1s 88ms/step - loss: 4.5002 - categorical_accuracy: 0.0270
Epoch 6/500
12/12 [==============================] - 1s 81ms/step - loss: 4.3089 - categorical_accuracy: 0.0595
Epoch 7/500
12/12 [==============================] - 1s 84ms/step - loss: 4.2043 - categorical_accuracy: 0.0514
Epoch 8/500
12/12 [==============================] - 1s 81ms/step - loss: 4.0198 - categorical_accuracy: 0.0514
Epoch 9/500
12/12 [==============================] - 1s 84ms/step - loss: 3.9911 - categorical_accuracy: 0.0649
Ep

In [ ]:
model.summary()

# 8. Make Predictions

In [86]:
res = model.predict(X_test)

1/1 [==============================] - 2s 2s/step


In [87]:
actions[np.argmax(res[0])]

'd'

In [88]:
actions[np.argmax(y_test[0])]

'm'

# 10. Evaluation using Confusion Matrix and Accuracy

In [92]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [93]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 304ms/step


In [94]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [95]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[19,  0],
        [ 1,  0]],

       [[14,  5],
        [ 0,  1]],

       [[19,  0],
        [ 1,  0]],

       [[ 5, 11],
        [ 3,  1]],

       [[19,  0],
        [ 1,  0]],

       [[19,  0],
        [ 1,  0]],

       [[19,  0],
        [ 1,  0]],

       [[19,  0],
        [ 1,  0]],

       [[18,  0],
        [ 2,  0]],

       [[16,  2],
        [ 2,  0]],

       [[18,  0],
        [ 2,  0]],

       [[18,  0],
        [ 2,  0]],

       [[19,  0],
        [ 1,  0]]])

In [96]:
accuracy_score(ytrue, yhat)

0.1

# 11. Test in Real Time

In [2]:
import numpy as np
import tensorflow as tf
from scipy import stats

2023-10-11 20:10:25.658980: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-11 20:10:26.042873: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-11 20:10:26.044508: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-11 20:10:28.310011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='hand_landmark.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
# Prepare input data (adjust to match your model's input shape)
input_data = np.random.rand(1, 224, 224, 3).astype(np.float32)

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame (resize, normalize, etc., to match model input)
    # You'll need to customize this part to match your model's input requirements
    frame = cv2.resize(frame, (input_shape[2], input_shape[1]))
    frame = frame / 255.0  # Normalize to [0, 1]
    input_data = np.expand_dims(frame, axis=0).astype(np.float32)

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Process the output data (use the model's output as needed)
    # You'll need to customize this part based on your model's output

    # Display the processed frame
    cv2.imshow('Inference', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

In [7]:
# Get the output tensor
output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])

print(output_data)

[[ 1.11490257e+02  1.74874237e+02 -5.92917437e-03  1.09697517e+02
   1.48955597e+02 -6.92485094e+00  1.11363617e+02  1.22920677e+02
  -1.34782972e+01  1.17257393e+02  1.04691582e+02 -1.83384438e+01
   1.20550980e+02  9.23995819e+01 -2.41702442e+01  1.07892799e+02
   9.59440231e+01 -2.07396736e+01  1.15661522e+02  7.73473587e+01
  -2.70041103e+01  1.20883293e+02  8.42512817e+01 -3.01766453e+01
   1.22152451e+02  9.19740982e+01 -3.12212486e+01  1.10198898e+02
   9.69804993e+01 -1.96090851e+01  1.19703331e+02  7.96363373e+01
  -2.57099075e+01  1.22695694e+02  9.21996689e+01 -2.84622765e+01
   1.21810280e+02  1.04376442e+02 -2.99171124e+01  1.12644928e+02
   1.02514145e+02 -1.81012611e+01  1.22929070e+02  8.77267380e+01
  -2.34097805e+01  1.24634956e+02  1.02215538e+02 -2.60374203e+01
   1.22681076e+02  1.17711571e+02 -2.66521111e+01  1.15152946e+02
   1.12055656e+02 -1.68184662e+01  1.23517418e+02  9.93238297e+01
  -2.03028164e+01  1.24931793e+02  1.08128441e+02 -2.21561527e+01
   1.23736

In [66]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [68]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(-1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti